In [1]:
import pandas as pd
from tqdm import tqdm
import pickle
import numpy as np
import os
from collections import defaultdict
import math

In [2]:
RAW_DATA_PATH = '/datasets/MIMIC-IV/physionet.org/files/mimiciv/2.2/'

# icu_data = pd.read_csv(os.path.join(RAW_DATA_PATH, 'icu/icustays.csv.gz'), compression='gzip', usecols=['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime'])
# icu_data = icu_data.loc[icu_data.intime.notna()]
# icu_data = icu_data.loc[icu_data.outtime.notna()]

# patients = pd.read_csv(os.path.join(RAW_DATA_PATH, 'hosp/patients.csv.gz'), compression='gzip', usecols=['subject_id', 'gender', 'anchor_age', 'anchor_year', 'dod'])
# patients['dob'] = patients.anchor_year - patients.anchor_age
# patients = patients.drop(columns=['anchor_year', 'anchor_age'])
# icu_data = icu_data.merge(patients, on='subject_id', how='left')
# icu_data['intime'] = pd.to_datetime(icu_data.intime)
# icu_data['outtime'] = pd.to_datetime(icu_data.outtime)
# icu_data['age'] = icu_data.intime.map(lambda x:x.year) - icu_data.dob.map(lambda x:x)
# icu_data = icu_data.loc[icu_data.age>=18]

# icu_data

In [3]:
if not os.path.exists('./data/features/chartevents.csv.gz'):
    output_file = 'filtered_data.csv.gz'  


    if os.path.exists(output_file):
        os.remove(output_file)

    for i, chunk in enumerate(tqdm(pd.read_csv(os.path.join(RAW_DATA_PATH, 'icu/chartevents.csv.gz'), 
                                compression='gzip', 
                                chunksize=10000000,
                                usecols=['stay_id','charttime','itemid','valuenum','valueuom']))):
        chunk = chunk.loc[chunk.stay_id.isin(icu_data.stay_id)]
        chunk = chunk.loc[chunk.charttime.notna() & chunk.valuenum.notna()].dropna()

        chunk.to_csv(output_file, mode='a', header=(i == 0), compression='gzip', index=False)
        del chunk

        # del chunk_merged
    ch = pd.read_csv(output_file, compression='gzip')
    print(ch.head())
    if os.path.exists(output_file):
        os.remove(output_file)

    ch.to_csv("./data/features/chartevents.csv.gz", compression='gzip', index=False)
else:
    ch = pd.read_csv('./data/features/chartevents.csv.gz', compression='gzip')
ch['table'] = 'chart'
print("===== READ CHARTEVENTS =====")

===== READ CHARTEVENTS =====


In [4]:
if not os.path.exists('./data/features/labevents.csv.gz'):
    output_file = 'filtered_data_lab.csv.gz'  

    if os.path.exists(output_file):
        os.remove(output_file)

    for i, chunk in enumerate(tqdm(pd.read_csv(os.path.join(RAW_DATA_PATH, 'hosp/labevents.csv.gz'), 
                                    compression='gzip', 
                                    chunksize=10000000,
                                    usecols=['hadm_id', 'itemid', 'charttime', 'value', 'valuenum', 'valueuom']))):
        chunk = chunk.loc[chunk.hadm_id.isin(icu_data.hadm_id)]
        chunk.hadm_id = chunk.hadm_id.astype(int)
        chunk = chunk.loc[chunk.charttime.notna()]
        chunk = chunk.loc[~(chunk.value.isna() & chunk.valuenum.isna())]
        chunk.to_csv(output_file, mode='a', header=(i == 0), compression='gzip', index=False)

        del chunk

    la = pd.read_csv(output_file, compression='gzip')
    print(la.head())
    if os.path.exists(output_file):
        os.remove(output_file)
    la.to_csv("./data/features/labevents.csv.gz", compression='gzip', index=False)
else:
    # pass
    la = pd.read_csv('./data/features/labevents.csv.gz', compression='gzip')
la['icustay_id'] = np.nan
la['table'] = 'lab'
print("===== READ LABEVENTS =====")

===== READ LABEVENTS =====


In [5]:
d_items = pd.read_csv(os.path.join(RAW_DATA_PATH, 'icu/d_items.csv.gz'), 
                      usecols=['itemid', 'label', 'abbreviation', 'category', 'unitname', 'param_type'], 
                      compression='gzip')
d_items

,itemid,label,abbreviation,category,unitname,param_type
0,220001,Problem List,Problem List,General,NaN,Text
1,220003,ICU Admission date,ICU Admission date,ADT,NaN,Date and time
2,220045,Heart Rate,HR,Routine Vital Signs,bpm,Numeric
3,220046,Heart rate Alarm - High,HR Alarm - High,Alarms,bpm,Numeric
4,220047,Heart Rate Alarm - Low,HR Alarm - Low,Alarms,bpm,Numeric
...,...,...,...,...,...,...
4009,230077,Documented on DC Summary (PICC),Documented on DC Summary (PICC),PICC Line Insertion,NaN,Text
4010,230078,Documented on DC Summary (Thora),Documented on DC Summary (Thora),Thoracentesis,NaN,Text
4011,230083,Replacement Fluid (Pre-Filter),Replacement Fluid (Pre-Filter),Dialysis,NaN,Text
4012,230084,Replacement Fluid (Post-Filter),Replacement Fluid (Post-Filter),Dialysis,NaN,Text


In [6]:
lab_items = pd.read_csv(os.path.join(RAW_DATA_PATH, 'hosp/d_labitems.csv.gz'), compression='gzip')
lab_items

,itemid,label,fluid,category
0,50801,Alveolar-arterial Gradient,Blood,Blood Gas
1,50802,Base Excess,Blood,Blood Gas
2,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas
3,50804,Calculated Total CO2,Blood,Blood Gas
4,50805,Carboxyhemoglobin,Blood,Blood Gas
...,...,...,...,...
1617,53150,Anti Hbs,Blood,Chemistry
1618,53151,Anti-la,Blood,Chemistry
1619,53152,HIV FINAL,Blood,Chemistry
1620,53153,HIV Screen,Blood,Chemistry


In [ ]:
labels = ['Blood Pressure', 'BP', 'Systolic', 'Diastolic']
pattern = '|'.join(labels)
bp_items = d_items[(d_items['unitname'] == 'mmHg') & 
                   (d_items['label'].str.contains(pattern, case=False, na=False)) & 
                   (d_items['category'] == 'Routine Vital Signs')]
bp_items = bp_items[bp_items['label'].str.contains('Systolic|Diastolic|Mean', case=False, na=False)]
bp_items = bp_items[bp_items['label'].str.contains('Arterial|Non Invasive|ART', case=False, na=False)]
sbp = bp_items[bp_items['label'].str.contains('Systolic', case=False, na=False)]['itemid'].tolist()
dbp = bp_items[bp_items['label'].str.contains('Diastolic', case=False, na=False)]['itemid'].tolist()
mbp = bp_items[bp_items['label'].str.contains('Mean', case=False, na=False)]['itemid'].tolist()
bp_items

In [ ]:
ch_bp = ch.loc[ch.itemid.isin(dbp+sbp+mbp)]
ch_bp = ch_bp.loc[(ch_bp.valuenum >= 0) & (ch_bp.valuenum <= 375)]
ch_bp.loc[ch_bp.itemid.isin(dbp), 'name'] = 'dbp'
ch_bp.loc[ch_bp.itemid.isin(dbp), 'name'] = 'dbp'
ch_bp.loc[ch_bp.itemid.isin(sbp), 'name'] = 'sbp'
ch_bp.loc[ch_bp.itemid.isin(mbp), 'name'] = 'mbp'
ch_bp['valueuom'] = 'mmHg'
ch_bp['value'] = None
events = ch_bp.copy()
del ch_bp
events

In [ ]:
gcs_items = d_items[d_items['label'].str.contains('gcs', case=False, na=False)]['itemid'].tolist()
gcs_items

In [ ]:
ch.loc[ch.itemid.isin(gcs_items)]

In [ ]:
hr_items = d_items[(d_items['label'].str.contains('heart rate', case=False, na=False)) & (d_items['category'] == 'Routine Vital Signs')]['itemid'].tolist()
hr_items

In [ ]:
ch_hr = ch.loc[ch.itemid.isin(hr_items)]
ch_hr = ch_hr.loc[(ch_hr.valuenum >= 0) & (ch_hr.valuenum <= 390)]
ch_hr['name'] = 'hr'
ch_hr['valueuom'] = 'bpm'
ch_hr['value'] = None
events = pd.concat([events, ch_hr])
del ch_hr
events

In [ ]:
rr_items = d_items[(d_items['label'].str.contains('respiratory rate', case=False, na=False)) & (d_items['category'] == 'Respiratory')]['itemid'].tolist()
rr_items

In [ ]:
ch_rr = ch.loc[ch.itemid.isin(rr_items)]
ch_rr = ch_rr.loc[(ch_rr.valuenum >= 0) & (ch_rr.valuenum <= 330)]
ch_rr['name'] = 'rr'
ch_rr['valueuom'] = 'insp/min'
ch_rr['value'] = None
events = pd.concat([events, ch_rr])
del ch_rr
events

In [ ]:
temp_items_c = d_items[(d_items['label'].str.contains('celsius', case=False, na=False))]['itemid'].tolist()
temp_items_f = d_items[(d_items['label'].str.contains('fahrenheit', case=False, na=False))]['itemid'].tolist()
temp_items_c, temp_items_f

In [ ]:
ch_temp_c = ch.loc[ch.itemid.isin(temp_items_c)]
ch_temp_f = ch.loc[ch.itemid.isin(temp_items_f)]
ch_temp_f.valuenum = (ch_temp_f.valuenum - 32) * 5 / 9
ch_temp = pd.concat([ch_temp_c, ch_temp_f])
del ch_temp_c
del ch_temp_f
ch_temp = ch_temp.loc[(ch_temp.valuenum >= 14.2) & (ch_temp.valuenum <= 47)]
ch_temp['name'] = 'temperature'
ch_temp['valueuom'] = '°C'
ch_temp['value'] = None
events = pd.concat([events, ch_temp])
del ch_temp
events

In [ ]:
weight_items_kg = d_items[(d_items['label'].str.contains('weight', case=False, na=False)) & 
                          (d_items['category'] == 'General') &
                          (d_items['unitname'] == 'kg')]['itemid'].tolist()
weight_items_lb = d_items[(d_items['label'].str.contains('weight', case=False, na=False)) & 
                          (d_items['category'] == 'General') &
                          (d_items['unitname'] != 'kg')]['itemid'].tolist()
weight_items_kg, weight_items_lb

In [ ]:
ch_w_kg = ch.loc[ch.itemid.isin(weight_items_kg)]
ch_w_lb = ch.loc[ch.itemid.isin(weight_items_lb)]
ch_w_lb.valuenum = ch_w_lb.valuenum * 0.453592
ch_w = pd.concat([ch_w_kg, ch_w_lb])
del ch_w_kg
del ch_w_lb
ch_w = ch_w.loc[(ch_w.valuenum >= 0) & (ch_w.valuenum <= 300)]
ch_w['name'] = 'weight'
ch_w['valueuom'] = 'kg'
ch_w['value'] = None
events = pd.concat([events, ch_w])
del ch_w
events

In [ ]:
height_events_in = d_items[(d_items['label'].str.contains('height', case=False, na=False)) &
                          (d_items['unitname'] == 'Inch')]['itemid'].tolist()
height_events_cm = d_items[(d_items['label'].str.contains('height', case=False, na=False)) &
                          (d_items['unitname'] == 'cm')]['itemid'].tolist()
height_events_in, height_events_cm

In [ ]:
ch_h_in = ch.loc[ch.itemid.isin(height_events_in)]
ch_h_cm = ch.loc[ch.itemid.isin(height_events_cm)]
ch_h_in.valuenum = ch_h_in.valuenum * 2.54
ch_h = pd.concat([ch_h_in, ch_h_cm])
del ch_h_in
del ch_h_cm
ch_h = ch_h.loc[(ch_h.valuenum >= 0) & (ch_h.valuenum <= 275)]
ch_h['name'] = 'height'
ch_h['valueuom'] = 'cm'
ch_h['value'] = None
events = pd.concat([events, ch_h])
del ch_h
events

In [ ]:
fio2 = d_items[d_items['label'].str.contains('Inspired O2 Fraction', case=False, na=False)]['itemid'].tolist()
fio2_extra = d_items[(d_items['label'].str.contains('fio2', case=False, na=False)) &
                    (d_items['category'] == 'ECMO')]['itemid'].tolist()
fio2 = fio2 + (fio2_extra)
fio2

In [ ]:
ch_fio2 = ch.loc[ch.itemid.isin(fio2)]
idx = ch_fio2.valuenum > 1.0
ch_fio2.loc[idx, 'valuenum'] = ch_fio2.loc[idx, 'valuenum'] / 100
ch_fio2 = ch_fio2.loc[(ch_fio2.valuenum>=0.2) & (ch_fio2.valuenum<=1)]
ch_fio2['name'] = 'fio2'
ch_fio2['valueuom'] = None
ch_fio2['value'] = None
events = pd.concat([events, ch_fio2])
del ch_fio2
events

In [ ]:
capillary_rate = d_items[d_items['label'].str.contains('Capillary', case=False, na=False)]
capillary_rate

In [ ]:
ch_cr = ch.loc[ch.itemid.isin(capillary_rate)]
ch_cr = ch_cr.loc[~(ch_cr.valuenum=='Other/Remarks')]
idx = (ch_cr.valuenum=='Normal <3 Seconds')|(ch_cr.valuenum=='Normal <3 secs')
ch_cr.loc[idx, 'valuenum'] = 0
idx = (ch_cr.valuenum=='Abnormal >3 Seconds')|(ch_cr.valuenum=='Abnormal >3 secs')
ch_cr.loc[idx, 'valuenum'] = 1
ch_cr['valueuom'] = None
ch_cr['name'] = 'crr'
events = pd.concat([events, ch_cr])
del ch_cr
events

In [ ]:
glucose_chart = d_items[(d_items['label'].str.contains('Glucose', case=False, na=False)) &
        (d_items['category'] == 'Labs') &
        (d_items['param_type'] == 'Numeric')]
glucose_lab = lab_items[(lab_items['label'].str.contains('Glucose', case=False, na=False)) &
                        (lab_items['fluid'] == 'Blood')]
gl_bl = [225664, 50809]
gl_wb = [226537, 52027]
gl_se = [220621, 50931]

In [ ]:
ev_blgl = pd.concat((ch.loc[ch.itemid.isin(gl_bl)], la.loc[la.itemid.isin(gl_bl)]))
ev_blgl = ev_blgl.loc[(ev_blgl.valuenum>=0)&(ev_blgl.valuenum<=2200)]
ev_blgl['name'] = 'glucose (blood)'
ev_wbgl = pd.concat((ch.loc[ch.itemid.isin(gl_wb)], la.loc[la.itemid.isin(gl_wb)]))
ev_wbgl = ev_wbgl.loc[(ev_wbgl.valuenum>=0)&(ev_wbgl.valuenum<=2200)]
ev_wbgl['name'] = 'glucose (whole blood)'
ev_segl = pd.concat((ch.loc[ch.itemid.isin(gl_se)], la.loc[la.itemid.isin(gl_se)]))
ev_segl = ev_segl.loc[(ev_segl.valuenum>=0)&(ev_segl.valuenum<=2200)]
ev_segl['name'] = 'glucose (serum)'

ev_gl = pd.concat([ev_blgl, ev_wbgl, ev_segl])
del ev_blgl, ev_wbgl, ev_segl

ev_gl['valueuom'] = 'mg/dL'
ev_gl['value'] = None
events = pd.concat([events, ev_gl])
del ev_gl
events

In [ ]:
bilirubin_chart = d_items[(d_items['label'].str.contains('bilirubin', case=False, na=False)) &
                          (d_items['category'] == 'Labs')]
bilirubin_lab = lab_items[lab_items['label'].str.contains('bilirubin', case=False, na=False)]
bilirubin_total = [225690, 50885]
bilirubin_direct = [225651, 50883]
bilirubin_indirect = [50884]

In [ ]:
ev_br = pd.concat((ch.loc[ch.itemid.isin(bilirubin_total+bilirubin_direct+bilirubin_indirect)], 
                   la.loc[la.itemid.isin(bilirubin_total+bilirubin_direct+bilirubin_indirect)]))
ev_br = ev_br.loc[(ev_br.valuenum>=0)&(ev_br.valuenum<=66)]
ev_br.loc[ev_br.itemid.isin(bilirubin_total), 'name'] = 'bilirubin (Total)'
ev_br.loc[ev_br.itemid.isin(bilirubin_direct), 'name'] = 'bilirubin (Direct)'
ev_br.loc[ev_br.itemid.isin(bilirubin_indirect), 'name'] = 'bilirubin (Indirect)'
ev_br['valueuom'] = 'mg/dL'
ev_br['value'] = None
events = pd.concat([events, ev_br])
del ev_br
events

In [ ]:
itb = [50812]
la_itb = la.loc[la.itemid.isin(itb)]
idx = (la_itb.value=='INTUBATED.')
la_itb.loc[idx, 'valuenum'] = 1
idx = (la_itb.value=='NOT INTUBATED.')
la_itb.loc[idx, 'valuenum'] = 0
la_itb['valueuom'] = None
la_itb['name'] = 'Intubated'
events = pd.concat([events, la_itb])
del la_itb
events

In [ ]:
o2sat = [50817, 220277]
sod = [50983, 50824]
pot = [50971, 50822]
mg = [50960]
po4 = [50970]
ca_total = [50893]
ca_free = [50808]
wbc = [51301, 51300]
hct = [50810, 51221]
hgb = [51222, 50811]
cl = [50902, 50806]
bic = [50882, 50803]
alt = [50861]
alp = [50863]
ast = [50878]
alb = [50862]
lac = [50813]
ld = [50954]
usg = [51498]
ph_ur = [51491, 51094, 220734]
ph_bl = [50820]
po2 = [50821]
pco2 = [50818]
tco2 = [50804]
be = [50802]
monos = [51254]
baso = [51146]
eos = [51200]
neuts = [51256]
lym_per = [51244, 51245]
lym_abs = [51133]
pt = [51274]
ptt = [51275]
inr = [51237]
agap = [50868]
bun = [51006]
cr_bl = [50912]
cr_ur = [51082]
mch = [51248]
mchc = [51249]
mcv = [51250]
rdw = [51277]
plt = [51265]
rbc = [51279]

In [ ]:
features = {'O2 Saturation': [o2sat, [0,100], '%'],
            'Sodium': [sod, [0,250], 'mEq/L'], 
            'Potassium': [pot, [0,15], 'mEq/L'], 
            'Magnesium': [mg, [0,22], 'mg/dL'], 
            'Phosphate': [po4, [0,22], 'mg/dL'],
            'Calcium Total': [ca_total, [0,40], 'mg/dL'],
            'Calcium Free': [ca_free, [0,10], 'mmol/L'],
            'WBC': [wbc, [0,1100], 'K/uL'], 
            'Hct': [hct, [0,100], '%'], 
            'Hgb': [hgb, [0,30], 'g/dL'], 
            'Chloride': [cl, [0,200], 'mEq/L'],
            'Bicarbonate': [bic, [0,66], 'mEq/L'],
            'ALT': [alt, [0,11000], 'IU/L'],
            'ALP': [alp, [0,4000], 'IU/L'],
            'AST': [ast, [0,22000], 'IU/L'],
            'Albumin': [alb, [0,10], 'g/dL'],
            'Lactate': [lac, [0,33], 'mmol/L'],
            'LDH': [ld, [0,35000], 'IU/L'],
            'SG Urine': [usg, [0,2], ''],
            'pH Urine': [ph_ur, [0,14], ''],
            'pH Blood': [ph_bl, [0,14], ''],
            'PO2': [po2, [0,770], 'mmHg'],
            'PCO2': [pco2, [0,220], 'mmHg'],
            'Total CO2': [tco2, [0,65], 'mEq/L'],
            'Base Excess': [be, [-31, 28], 'mEq/L'],
            'Monocytes': [monos, [0,100], '%'],
            'Basophils': [baso, [0,100], '%'],
            'Eoisinophils': [eos, [0,100], '%'],
            'Neutrophils': [neuts, [0,100], '%'],
            'Lymphocytes': [lym_per, [0,100], '%'],
            'Lymphocytes (Absolute)': [lym_abs, [0,25000], '#/uL'],
            'PT': [pt, [0,150], 'sec'],
            'PTT': [ptt, [0,150], 'sec'],
            'INR': [inr, [0,150], ''],
            'Anion Gap': [agap, [0,55], 'mg/dL'],
            'BUN': [bun, [0,275], 'mEq/L'],
            'Creatinine Blood': [cr_bl, [0,66], 'mg/dL'],
            'Creatinine Urine': [cr_ur, [0,650], 'mg/dL'],
            'MCH': [mch, [0,50], 'pg'],
            'MCHC': [mchc, [0,50], '%'],
            'MCV': [mcv, [0,150], 'fL'],
            'RDW': [rdw, [0,37], '%'],
            'Platelet Count': [plt, [0,2200], 'K/uL'],
            'RBC': [rbc, [0,14], 'm/uL']
            }

In [ ]:
for k, v in features.items():
    print (k)
    ev_k = pd.concat((ch.loc[ch.itemid.isin(v[0])], la.loc[la.itemid.isin(v[0])]))
    ev_k = ev_k.loc[(ev_k.valuenum>=v[1][0])&(ev_k.valuenum<=v[1][1])]
    ev_k['name'] = k
    ev_k['valueuom'] = v[2]
    ev_k['value'] = None
    assert (ev_k.valuenum.isna().sum()==0)
    events = pd.concat([events, ev_k])
    del ev_k
events

In [ ]:
# os.makedirs('./data/events/', exist_ok=True)
# events.to_csv('./data/events/events.csv.gz', compression='gzip', index=False)

In [ ]:
# if os.path.exists('./data/events/events.csv.gz'):
#     events = pd.read_csv('./data/events/events.csv.gz', compression='gzip')
del ch, la

In [ ]:
oe = pd.read_csv(os.path.join(RAW_DATA_PATH, 'icu/outputevents.csv.gz'), 
                 usecols = ['stay_id', 'itemid', 'charttime', 'value', 'valueuom'],
                 compression='gzip')
oe = oe.loc[oe.value.notna()]
oe['valuenum'] = oe.value
oe.value = None
oe = oe.loc[oe.stay_id.isin(icu_data.stay_id)]
oe.stay_id = oe.stay_id.astype(int)
oe['table'] = 'output'
oe

In [ ]:
d_items.loc[d_items.label.isna(), 'label'] = ''
d_items.LABEL = d_items.label.str.lower()
oeitems = oe[['itemid']].drop_duplicates()
oeitems = oeitems.merge(d_items, on='itemid', how='left')
oeitems

In [ ]:
uf = [40286]
keys = ['urine', 'foley', 'void', 'nephrostomy', 'condom', 'drainage bag']
cond = pd.concat([oeitems.label.str.contains(k) for k in keys], axis=1).any(axis='columns')
ur = list(oeitems.loc[cond].itemid)
keys = ['stool', 'fecal', 'colostomy', 'ileostomy', 'rectal']
cond = pd.concat([oeitems.label.str.contains(k) for k in keys], axis=1).any(axis='columns')
st = list(oeitems.loc[cond].itemid)
ct = list(oeitems.loc[oeitems.label.str.contains('chest tube')].itemid) + [226593, 226590, 226591, 226595, 226592]
gs = [40059, 40052, 226576, 226575, 226573, 40051, 226630]
ebl = [40064, 226626, 40491, 226629]
em = [40067, 226571, 40490, 41015, 40427]
jp = list(oeitems.loc[oeitems.label.str.contains('jackson')].itemid)
res = [227510, 227511, 42837, 43892, 44909, 44959]
pre = [40060, 226633]

In [ ]:
features = {'Ultrafiltrate': [uf, [0,7000],'mL'],
            'Urine': [ur, [0,2500], 'mL'],
            'Stool': [st, [0,4000], 'mL'],
            'Chest Tube': [ct, [0,2500], 'mL'],
            'Gastric': [gs, [0,4000], 'mL'],
            'EBL': [ebl, [0,10000], 'mL'],
#             'Pre-admission': [pre, [0,13000], 'mL'], # Repeated by mistake.
            'Emesis': [em, [0,2000], 'mL'],
            'Jackson-Pratt': [jp, [0,2000], 'ml'],
            'Residual': [res, [0, 1050], 'mL'],
            'Pre-admission Output': [pre, [0, 13000], 'ml']
            }

In [ ]:
for k, v in features.items():
    print (k)
    ev_k = oe.loc[oe.itemid.isin(v[0])]
    ind = (ev_k.valuenum>=v[1][0])&(ev_k.valuenum<=v[1][1])
    med = ev_k.valuenum.loc[ind].median()
    ev_k.loc[~ind, 'valuenum'] = med
    ev_k['name'] = k
    ev_k['valueuom'] = v[2]
    events = pd.concat([events, ev_k])
    del ev_k
events

In [ ]:
ie = pd.read_csv(os.path.join(RAW_DATA_PATH, 'icu/inputevents.csv.gz'), 
                 usecols=['stay_id', 'itemid', 'starttime', 'endtime', 'amount', 'amountuom'],
                 compression='gzip')
ie['table'] = 'input_events'
ie = ie.loc[ie.amount.notna()]
ie = ie.loc[ie.stay_id.isin(icu_data.stay_id)]
ie.starttime = pd.to_datetime(ie.starttime)
ie.endtime = pd.to_datetime(ie.endtime)
ie['TD'] = ie.endtime - ie.starttime
new_ie = ie.loc[ie.TD<=pd.Timedelta(1,'h')].drop(columns=['starttime', 'TD'])
ie = ie.loc[ie.TD>pd.Timedelta(1,'h')]
ie

In [ ]:
new_rows = []
for _,row in tqdm(ie.iterrows()):
    icuid, iid, amo, uom, stm, td = row.stay_id, row.itemid, row.amount, row.amountuom, row.starttime, row.TD
    td = td.total_seconds()/60
    num_hours = td // 60
    hour_amount = 60*amo/td
    for i in range(1,int(num_hours)+1):
        new_rows.append([icuid, iid, stm+pd.Timedelta(i,'h'), hour_amount, uom])
    rem_mins = td % 60
    if rem_mins>0:
        new_rows.append([icuid, iid, row['endtime'], rem_mins*amo/td, uom])
new_rows = pd.DataFrame(new_rows, columns=['stay_id', 'itemid', 'endtime', 'amount', 'amountuom'])
new_ie = pd.concat((new_ie, new_rows))
ie = new_ie.copy()
del new_ie
ie.rename(columns={'endtime':'charttime'}, inplace=True)

In [ ]:
ie.rename(columns={'amount':'valuenum', 'amountuom':'valueuom'}, inplace=True)
events.charttime = pd.to_datetime(events.charttime)

In [ ]:
ind = (ie.valueuom=='mcg')
ie.loc[ind, 'valuenum'] = ie.loc[ind, 'valuenum']*0.001
ie.loc[ind, 'valueuom'] = 'mg'
ind = (ie.valueuom=='L')
ie.loc[ind, 'valuenum'] = ie.loc[ind, 'valuenum']*1000
ie.loc[ind, 'valueuom'] = 'ml'
ie

In [ ]:
os.makedirs('./data/ie/', exist_ok=True)
ie.to_csv('./data/ie/ie.csv.gz', compression='gzip', index=False)

In [ ]:
if os.path.exists('./data/ie/ie.csv.gz'):
    ie = pd.read_csv('./data/ie/ie.csv.gz', compression='gzip')
ie

In [ ]:
vaso = [30051, 222315]
ev_vaso = ie.loc[ie.itemid.isin(vaso)]
ind1 = (ev_vaso.valuenum==0)
ind2 = ev_vaso.valueuom.isin(['U','units'])
ind3 = (ev_vaso.valuenum>=0)&(ev_vaso.valuenum<=400)
ind = ((ind2&ind3)|ind1)
med = ev_vaso.valuenum.loc[ind].median()
ev_vaso.loc[~ind, 'valuenum'] = med
ev_vaso['valueuom'] = 'units'
ev_vaso['name'] = 'Vasopressin'
events = pd.concat([events, ev_vaso])
del ev_vaso
# events.to_csv('./data/events/events.csv.gz', compression='gzip', index=False)

In [ ]:
# if os.path.exists('./data/events/events.csv.gz'):
#     events = pd.read_csv('./data/events/events.csv.gz', compression='gzip')

In [ ]:
vanc = [225798]
ev_vanc = ie.loc[ie.itemid.isin(vanc)]
ind = ev_vanc.valueuom.isin(['mg'])
ev_vanc.loc[ind, 'valuenum'] = ev_vanc.loc[ind, 'valuenum']*0.001 
ev_vanc['valueuom'] = 'g'
ind = (ev_vanc.valuenum>=0)&(ev_vanc.valuenum<=8)
med = ev_vanc.valuenum.loc[ind].median()
ev_vanc.loc[~ind, 'valuenum'] = med
ev_vanc['name'] = 'Vacomycin'
events = pd.concat([events, ev_vanc])
del ev_vanc
events

In [ ]:
cagl = [30023, 221456, 227525, 42504, 43070, 45699, 46591, 44346, 46291]
ev_cagl = ie.loc[ie.itemid.isin(cagl)]
ind = ev_cagl.valueuom.isin(['mg'])
ev_cagl.loc[ind, 'valuenum'] = ev_cagl.loc[ind, 'valuenum']*0.001 
ind1 = (ev_cagl.valuenum==0)
ind2 = ev_cagl.valueuom.isin(['mg', 'gm', 'grams'])
ind3 = (ev_cagl.valuenum>=0)&(ev_cagl.valuenum<=200)
ind = (ind2&ind3)|ind1
med = ev_cagl.valuenum.loc[ind].median()
ev_cagl.loc[~ind, 'valuenum'] = med
ev_cagl['valueuom'] = 'g'
ev_cagl['name'] = 'Calcium Gluconate'
events = pd.concat([events, ev_cagl])
del ev_cagl
events

In [ ]:
furo = [30123, 221794, 228340]
ev_furo = ie.loc[ie.itemid.isin(furo)]
ind1 = (ev_furo.valuenum==0)
ind2 = (ev_furo.valueuom=='mg')
ind3 = (ev_furo.valuenum>=0)&(ev_furo.valuenum<=250)
ind = ind1|(ind2&ind3)
med = ev_furo.valuenum.loc[ind].median()
ev_furo.loc[~ind, 'valuenum'] = med
ev_furo['valueuom'] = 'mg'
ev_furo['name'] = 'Furosemide'
events = pd.concat([events, ev_furo])
del ev_furo
events

In [ ]:
famo = [225907]
ev_famo = ie.loc[ie.itemid.isin(famo)]
ind1 = (ev_famo.valuenum==0)
ind2 = (ev_famo.valueuom=='dose')
ind3 = (ev_famo.valuenum>=0)&(ev_famo.valuenum<=1)
ind = ind1|(ind2&ind3)
med = ev_famo.valuenum.loc[ind].median()
ev_famo.loc[~ind, 'valuenum'] = med
ev_famo['valueuom'] = 'dose'
ev_famo['name'] = 'Famotidine'
events = pd.concat([events, ev_famo])
del ev_famo
events

In [ ]:
pipe = [225893, 225892]
ev_pipe = ie.loc[ie.itemid.isin(pipe)]
ind1 = (ev_pipe.valuenum==0)
ind2 = (ev_pipe.valueuom=='dose')
ind3 = (ev_pipe.valuenum>=0)&(ev_pipe.valuenum<=1)
ind = ind1|(ind2&ind3)
med = ev_pipe.valuenum.loc[ind].median()
ev_pipe.loc[~ind, 'valuenum'] = med
ev_pipe['valueuom'] = 'dose'
ev_pipe['name'] = 'Piperacillin'
events = pd.concat([events, ev_pipe])
del ev_pipe
events

In [ ]:
cefa = [225850]
ev_cefa = ie.loc[ie.itemid.isin(cefa)]
ind1 = (ev_cefa.valuenum==0)
ind2 = (ev_cefa.valueuom=='dose')
ind3 = (ev_cefa.valuenum>=0)&(ev_cefa.valuenum<=2)
ind = ind1|(ind2&ind3)
med = ev_cefa.valuenum.loc[ind].median()
ev_cefa.loc[~ind, 'valuenum'] = med 
ev_cefa['valueuom'] = 'dose'
ev_cefa['name'] = 'Cefazolin'
events = pd.concat([events, ev_cefa])
del ev_cefa
events

In [ ]:
fibe = [225936, 30166, 30073, 227695, 30088, 225928, 226051, 
        226050, 226048, 45381, 45597, 227699, 227696, 44218, 
        45406, 44675, 226049, 44202, 45370, 227698, 226027, 
        42106, 43994, 45865, 44318, 42091, 44699, 44010, 43134, 
        44045, 43088, 42641, 45691, 45515, 45777, 42663, 42027, 
        44425, 45657, 45775, 44631, 44106, 42116, 44061, 44887, 
        42090, 42831, 45541, 45497, 46789, 44765, 42050]
ev_fibe = ie.loc[ie.itemid.isin(fibe)]
ind1 = (ev_fibe.valuenum==0)
ind2 = (ev_fibe.valueuom=='ml')
ind3 = (ev_fibe.valuenum>=0)&(ev_fibe.valuenum<=1600)
ind = ind1|(ind2&ind3)
med = ev_fibe.valuenum.loc[ind].median()
ev_fibe.loc[~ind, 'valuenum'] = med 
ev_fibe['name'] = 'Fiber'
ev_fibe['valueuom'] = 'ml'
events = pd.concat([events, ev_fibe])
del ev_fibe
events

In [ ]:
pant = [225910, 40549, 41101, 41583, 44008, 40700, 40550]
ev_pant = ie.loc[ie.itemid.isin(pant)]
ind = (ev_pant.valuenum>0)
ev_pant.loc[ind, 'valuenum'] = 1
ind = (ev_pant.valuenum>=0)
med = ev_pant.valuenum.loc[ind].median()
ev_pant.loc[~ind, 'valuenum'] = med
ev_pant['name'] = 'Pantoprazole'
ev_pant['valueuom'] = 'dose'
events = pd.concat([events, ev_pant])
del ev_pant
events

In [ ]:
masu = [222011, 30027, 227524]
ev_masu = ie.loc[ie.itemid.isin(masu)]
ind = (ev_masu.valueuom=='mg')
ev_masu.loc[ind, 'valuenum'] = ev_masu.loc[ind, 'valuenum']*0.001
ind1 = (ev_masu.valuenum==0)
ind2 = ev_masu.valueuom.isin(['gm', 'grams', 'mg'])
ind3 = (ev_masu.valuenum>=0)&(ev_masu.valuenum<=125)
ind = ind1|(ind2&ind3)
med = ev_masu.valuenum.loc[ind].median()
ev_masu.loc[~ind, 'valuenum'] = med 
ev_masu['valueuom'] = 'g'
ev_masu['name'] = 'Magnesium Sulphate'
events = pd.concat([events, ev_masu])
del ev_masu
events

In [ ]:
poch = [30026, 225166, 227536]
ev_poch = ie.loc[ie.itemid.isin(poch)]
ind1 = (ev_poch.valuenum==0)
ind2 = ev_poch.valueuom.isin(['mEq', 'mEq.'])
ind3 = (ev_poch.valuenum>=0)&(ev_poch.valuenum<=501)
ind = ind1|(ind2&ind3)
med = ev_poch.valuenum.loc[ind].median()
ev_poch.loc[~ind, 'valuenum'] = med 
ev_poch['valueuom'] = 'mEq'
ev_poch['name'] = 'KCl'
events = pd.concat([events, ev_poch])
del ev_poch
events

In [ ]:
mida = [30124, 221668]
prop = [30131, 222168]
albu25 = [220862, 30009]
albu5 = [220864, 30008]
ffpl = [30005, 220970]
lora = [30141, 221385]
mosu = [30126, 225154]
game = [30144, 225799]
lari = [30021, 225828]
milr = [30125, 221986]
crys = [30101, 226364, 30108, 226375]
hepa = [30025, 225975, 225152]
prbc = [30001, 225168, 30104, 226368, 227070]
poin = [30056, 226452, 30109, 226377]
neos = [30128, 221749, 30127]
pigg = [226089, 30063]
nigl = [30121, 222056, 30049]
nipr = [30050, 222051]
meto = [225974]
nore = [30120, 221906, 30047]
coll = [30102, 226365, 30107, 226376]
hyzi = [221828]
gtfl = [226453, 30059]
hymo = [30163, 221833]
fent = [225942, 30118, 221744, 30149]
inre = [30045, 223258, 30100]
inhu = [223262]
ingl = [223260]
innp = [223259]
nana = [30140]
d5wa = [30013, 220949]
doth = [30015, 225823, 30060, 225825, 220950, 30016, 
        30061, 225827, 225941, 30160, 220952, 30159, 
        30014, 30017, 228142, 228140, 45360, 228141, 
        41550]
nosa = [225158, 30018]
hans = [30020, 225159]
stwa = [225944, 30065]
frwa = [30058, 225797, 41430, 40872, 41915, 43936, 
        41619, 42429, 44492, 46169, 42554]
solu = [225943]
dopa = [30043, 221662]
epin = [30119, 221289, 30044]
amio = [30112, 221347, 228339, 45402]
tpnu = [30032, 225916, 225917, 30096]
msbo = [227523]
pcbo = [227522]
prad = [30054, 226361]

In [ ]:
features = {'Midazolam': [mida, [0, 500], 'mg'],
            'Propofol': [prop, [0, 12000], 'mg'],
            'Albumin 25%': [albu25, [0, 750], 'ml'],
            'Albumin 5%': [albu5, [0, 1300], 'ml'],
            'Fresh Frozen Plasma': [ffpl, [0, 33000], 'ml'],
            'Lorazepam': [lora, [0, 300], 'mg'],
            'Morphine Sulfate': [mosu, [0, 4000], 'mg'],
            'Gastric Meds': [game, [0, 7000], 'ml'],
            'Lactated Ringers': [lari, [0, 17000], 'ml'],
            'Milrinone': [milr, [0, 50], 'ml'],
            'OR/PACU Crystalloid': [crys, [0, 22000], 'ml'],
            'Packed RBC': [prbc, [0, 17250], 'ml'],
            'PO intake': [poin, [0, 11000], 'ml'],
            'Neosynephrine': [neos, [0, 1200], 'mg'],
            'Piggyback': [pigg, [0, 1000], 'ml'],
            'Nitroglycerine': [nigl, [0, 350], 'mg'],
            'Nitroprusside': [nipr, [0, 430], 'mg'],
            'Metoprolol': [meto, [0, 151], 'mg'],
            'Norepinephrine': [nore, [0, 80], 'mg'],
            'Colloid': [coll, [0, 20000], 'ml'],
            'Hydralazine': [hyzi, [0, 80], 'mg'],
            'GT Flush': [gtfl, [0, 2100], 'ml'],
            'Hydromorphone': [hymo, [0, 125], 'mg'],
            'Fentanyl': [fent, [0, 20], 'mg'],
            'Insulin Regular': [inre, [0, 1500], 'units'],
            'Insulin Humalog': [inhu, [0, 340], 'units'],
            'Insulin largine': [ingl, [0, 150], 'units'],
           }

features_remaining = {
            'Insulin NPH': [innp, [0, 100], 'units'],
            'Unknown': [nana, [0, 1100], 'ml'],
            'D5W': [d5wa, [0,11000], 'ml'],
            'Dextrose Other': [doth, [0,4000], 'ml'],
            'Normal Saline': [nosa, [0, 11000], 'ml'],
            'Half Normal Saline': [hans, [0, 2000], 'ml'],
            'Sterile Water': [stwa, [0, 10000], 'ml'],
            'Free Water': [frwa, [0, 2500], 'ml'],
            'Solution': [solu, [0, 1500], 'ml'],
            'Dopamine': [dopa, [0, 1300], 'mg'],
            'Epinephrine': [epin, [0, 100], 'mg'],
            'Amiodarone': [amio, [0, 1200], 'mg'],
            'TPN': [tpnu, [0, 1600], 'ml'],
            'Magnesium Sulfate (Bolus)': [msbo, [0, 250], 'ml'],
            'KCl (Bolus)': [pcbo, [0, 500], 'ml'],
            'Pre-admission Intake': [prad, [0, 30000], 'ml']
            }

In [ ]:
for k, v in features.items():
    print (k)
    ev_k = ie.loc[ie.itemid.isin(v[0])]
    ind = (ev_k.valuenum>=v[1][0])&(ev_k.valuenum<=v[1][1])
    med = ev_k.valuenum.loc[ind].median()
    ev_k.loc[~ind, 'valuenum'] = med
    ev_k['name'] = k
    ev_k['valueuom'] = v[2]
    events = pd.concat([events, ev_k])
    del ev_k

In [ ]:
os.makedirs('./data/events/', exist_ok=True)
events.to_csv('./data/events/events.csv.gz', compression='gzip', index=False)
events

In [ ]:
if os.path.exists('./data/events/events.csv.gz'):
    events = pd.read_csv('./data/events/events.csv.gz', compression='gzip')
events

In [ ]:
for k, v in features_remaining.items():
    print (k)
    ev_k = ie.loc[ie.itemid.isin(v[0])]
    ind = (ev_k.valuenum>=v[1][0])&(ev_k.valuenum<=v[1][1])
    med = ev_k.valuenum.loc[ind].median()
    ev_k.loc[~ind, 'valuenum'] = med
    ev_k['name'] = k
    ev_k['valueuom'] = v[2]
    events = pd.concat([events, ev_k])
    del ev_k

In [ ]:
if os.path.exists('./data/events/events.csv.gz'):
    events = pd.read_csv('./data/events/events.csv.gz', compression='gzip')
events

In [ ]:
ev_k = ie.loc[ie.itemid.isin(hepa)]
ind1 = ev_k.valueuom.isin(['U', 'units'])
ind2 = (ev_k.valuenum>=0)&(ev_k.valuenum<=25300)
ind = (ind1&ind2)
med = ev_k.valuenum.loc[ind].median()
ev_k.loc[~ind, 'valuenum'] = med
ev_k['name'] = 'Heparin'
ev_k['valueuom'] = 'units'
events = pd.concat([events, ev_k])
del ev_k
events

In [ ]:
ie_mv = pd.read_csv(os.path.join(RAW_DATA_PATH,'icu/inputevents.csv.gz'), 
                    usecols = ['stay_id', 'starttime', 'patientweight'],
                   compression='gzip')
ie_mv = ie_mv.drop_duplicates()
ie_mv = ie_mv.loc[ie_mv.stay_id.isin(icu_data.stay_id)]
ie_mv.rename(columns={'starttime':'charttime', 'patientweight':'valuenum'}, inplace=True)
ie_mv = ie_mv.loc[(ie_mv.valuenum>=0)&(ie_mv.valuenum<=300)]
ie_mv['valueuom'] = 'kg'
ie_mv['name'] = 'Weight'
events = pd.concat([events, ie_mv])[['hadm_id', 'stay_id', 'charttime', 'valuenum', 'table', 'name']]
del ie_mv

In [ ]:
events.charttime = pd.to_datetime(events.charttime)
icu_data.intime = pd.to_datetime(icu_data.intime)
icu_data.outtime = pd.to_datetime(icu_data.outtime)

In [ ]:
icu_data

In [ ]:
icu_data['icustay_times'] = icu_data.apply(lambda x:[x.stay_id, x.intime, x.outtime], axis=1)
icu_data

In [ ]:
adm_icu_times = icu_data.groupby('hadm_id').agg({'icustay_times':list}).reset_index()
adm_icu_times

In [ ]:
icu_data.drop(columns=['icustay_times'], inplace=True)
events = events.merge(adm_icu_times, on=['hadm_id'], how='left')
events

In [ ]:
idx = events.stay_id.isna()
tqdm.pandas()
def f(x):
    chart_time = x.charttime
    for icu_times in x.icustay_times:
        if icu_times[1]<=chart_time<=icu_times[2]:
            return icu_times[0]
events.loc[idx, 'stay_id'] = (events.loc[idx]).progress_apply(f, axis=1)
events.drop(columns=['icustay_times'], inplace=True)
events = events.loc[events.stay_id.notna()]
events.drop(columns=['hadm_id'], inplace=True)
events

In [ ]:
icu_data = icu_data.loc[icu_data.stay_id.isin(events.stay_id)]

In [ ]:
events = events.merge(icu_data[['stay_id', 'intime']], on='stay_id', how='left')
events['rel_charttime'] = events.charttime-events.intime
events.drop(columns=['intime', 'charttime'], inplace=True)
events.rel_charttime = events.rel_charttime.dt.total_seconds()//60

In [9]:
icu_full = icu_data.copy()

In [ ]:
icu_data = icu_data.loc[(icu_data.outtime-icu_data.intime)>=pd.Timedelta(24,'h')]

In [ ]:
adm = pd.read_csv(os.path.join(RAW_DATA_PATH,'hosp/admissions.csv.gz'),
                  usecols=['hadm_id', 'deathtime'],
                  compression='gzip')
icu_data = icu_data.merge(adm, on='hadm_id', how='left')

In [ ]:
icu_data.deathtime = pd.to_datetime(icu_data.deathtime)
icu_data = icu_data.loc[((icu_data.deathtime-icu_data.intime)>=pd.Timedelta(24,'h'))|icu_data.deathtime.isna()]

In [ ]:
icu_data = icu_data.loc[icu_data.stay_id.isin(events.loc[events.rel_charttime<24*60].stay_id)]

In [ ]:
os.makedirs('./data/icu_data/', exist_ok=True)
icu_data.to_csv('./data/icu_data/icu_data.csv.gz', compression='gzip', index=False)

In [7]:
if os.path.exists('./data/icu_data/icu_data.csv.gz'):
    icu_data = pd.read_csv('./data/icu_data/icu_data.csv.gz', compression='gzip')
icu_data

,subject_id,hadm_id,stay_id,intime,outtime,gender,dod,dob,age,deathtime_x,deathtime_y,deathtime
0,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,F,NaN,2102,55,NaN,NaN,NaN
1,10001725,25563031,31205490,2110-04-11 15:52:22,2110-04-12 23:59:56,F,NaN,2064,46,NaN,NaN,NaN
2,10001884,26184834,37510196,2131-01-11 04:20:05,2131-01-20 08:27:30,F,2131-01-20,2054,77,2131-01-20 05:15:00,2131-01-20 05:15:00,2131-01-20 05:15:00
3,10002013,23581541,39060235,2160-05-18 10:00:53,2160-05-19 17:33:33,F,NaN,2103,57,NaN,NaN,NaN
4,10002155,23822395,33685454,2129-08-04 12:45:00,2129-08-10 17:02:38,F,2131-03-10,2048,81,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
57530,19999297,21439025,37364566,2162-08-16 05:48:32,2162-08-23 06:22:41,M,2162-08-23,2120,42,2162-08-23 04:16:00,2162-08-23 04:16:00,2162-08-23 04:16:00
57531,19999442,26785317,32336619,2148-11-19 14:23:43,2148-11-26 13:12:15,M,NaN,2105,43,NaN,NaN,NaN
57532,19999828,25744818,36075953,2149-01-08 18:12:00,2149-01-10 13:11:02,F,NaN,2101,48,NaN,NaN,NaN
57533,19999840,21033226,38978960,2164-09-12 09:26:28,2164-09-17 16:35:15,M,2164-09-17,2106,58,2164-09-17 13:42:00,2164-09-17 13:42:00,2164-09-17 13:42:00


In [8]:
if os.path.exists('./data/events/events.csv.gz'):
    events = pd.read_csv('./data/events/events.csv.gz', compression='gzip')
events

/pkgs/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,stay_id,valuenum,table,name,rel_charttime
0,39553978.0,82.0,chart,sbp,421.0
1,39553978.0,59.0,chart,dbp,421.0
2,39553978.0,63.0,chart,mbp,421.0
3,39553978.0,85.0,chart,sbp,480.0
4,39553978.0,55.0,chart,dbp,480.0
...,...,...,...,...,...
83384656,36195440.0,60.0,NaN,Weight,26.0
83384657,36195440.0,60.0,NaN,Weight,29.0
83384658,36195440.0,60.0,NaN,Weight,39.0
83384659,36195440.0,94.0,NaN,Weight,2736.0


In [10]:
all_icustays = np.array(icu_full.stay_id)
sup_icustays = np.array(icu_data.stay_id)
unsup_icustays = np.setdiff1d(all_icustays, sup_icustays)
all_icustays = np.concatenate((sup_icustays, unsup_icustays), axis=-1)

In [11]:
events.rename(columns={'rel_charttime':'minute', 'name':'variable', 
                       'valuenum':'value', 'stay_id':'ts_id'}, inplace=True)
events

,ts_id,value,table,variable,minute
0,39553978.0,82.0,chart,sbp,421.0
1,39553978.0,59.0,chart,dbp,421.0
2,39553978.0,63.0,chart,mbp,421.0
3,39553978.0,85.0,chart,sbp,480.0
4,39553978.0,55.0,chart,dbp,480.0
...,...,...,...,...,...
83384656,36195440.0,60.0,NaN,Weight,26.0
83384657,36195440.0,60.0,NaN,Weight,29.0
83384658,36195440.0,60.0,NaN,Weight,39.0
83384659,36195440.0,94.0,NaN,Weight,2736.0


In [12]:
icu_full.rename(columns={'stay_id':'ts_id'}, inplace=True)
data_age = icu_full[['ts_id', 'age']]
data_age['variable'] = 'Age'
data_age.rename(columns={'age':'value'}, inplace=True)
data_gen = icu_full[['ts_id', 'gender']]
data_gen.loc[data_gen.gender=='M', 'gender'] = 0
data_gen.loc[data_gen.gender=='F', 'gender'] = 1
data_gen['variable'] = 'Gender'
data_gen.rename(columns={'gender':'value'}, inplace=True)
data = pd.concat((data_age, data_gen), ignore_index=True)
data['minute'] = 0
events = pd.concat((data, events), ignore_index=True)
events

/pkgs/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/pkgs/pytorch-1.7.1-cuda11.0-conda-python3.6/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/pkgs/pytorch-1.7.1-cuda11.0-conda-python3.6/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

,ts_id,value,variable,minute,table
0,37067082.0,55,Age,0.0,NaN
1,31205490.0,46,Age,0.0,NaN
2,37510196.0,77,Age,0.0,NaN
3,39060235.0,57,Age,0.0,NaN
4,33685454.0,81,Age,0.0,NaN
...,...,...,...,...,...
83499726,36195440.0,60,Weight,26.0,NaN
83499727,36195440.0,60,Weight,29.0,NaN
83499728,36195440.0,60,Weight,39.0,NaN
83499729,36195440.0,94,Weight,2736.0,NaN


In [13]:
# Use a set to store unique rows as hashes
chunk_size = 10_000_000
unique_rows = set()
final_data = []

for i in tqdm(range(0, len(events), chunk_size), desc="Processing Chunks"):
    chunk = events[i:i + chunk_size]
    for row in chunk.itertuples(index=False, name=None):
        if row not in unique_rows:
            unique_rows.add(row)
            final_data.append(row)

Processing Chunks: 100%|██████████| 9/9 [01:51<00:00, 12.38s/it]


In [14]:
# Convert back to a DataFrame
events = pd.DataFrame(final_data, columns=events.columns)
events

,ts_id,value,variable,minute,table
0,37067082.0,55.0,Age,0.0,NaN
1,31205490.0,46.0,Age,0.0,NaN
2,37510196.0,77.0,Age,0.0,NaN
3,39060235.0,57.0,Age,0.0,NaN
4,33685454.0,81.0,Age,0.0,NaN
...,...,...,...,...,...
82876661,36195440.0,60.0,Weight,26.0,NaN
82876662,36195440.0,60.0,Weight,29.0,NaN
82876663,36195440.0,60.0,Weight,39.0,NaN
82876664,36195440.0,94.0,Weight,2736.0,NaN


In [ ]:
adm = pd.read_csv(os.path.join(RAW_DATA_PATH,'hosp/admissions.csv.gz'))
                  #usecols=['hadm_id', 'hospital_expire_flag'])
# oc = icu_full[['ts_id', 'hadm_id', 'subject_id']].merge(adm, on='hadm_id', how='left')
# oc = oc.rename(columns={'hospital_expire_flag': 'in_hospital_mortality'})
adm

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431226,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaN,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0
431227,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,NaN,EW EMER.,P16C7J,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2147-07-17 17:18:00,2147-07-18 17:34:00,0
431228,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,P58A9J,EMERGENCY ROOM,DIED,Other,ENGLISH,WIDOWED,WHITE,2164-09-10 11:09:00,2164-09-10 14:46:00,1
431229,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,NaN,EW EMER.,P506DE,EMERGENCY ROOM,HOME,Other,ENGLISH,WIDOWED,WHITE,2164-07-24 21:16:00,2164-07-25 01:20:00,0


In [16]:
all_sup_subjects = icu_data.subject_id.unique()
np.random.seed(0)
np.random.shuffle(all_sup_subjects)
S = len(all_sup_subjects)
bp1, bp2 = int(0.64*S), int(0.8*S)
train_sub = all_sup_subjects[:bp1]
valid_sub = all_sup_subjects[bp1:bp2]
test_sub = all_sup_subjects[bp2:]
icu_data.rename(columns={'stay_id':'ts_id'}, inplace=True)
train_ids = np.array(icu_data.loc[icu_data.subject_id.isin(train_sub)].ts_id)
valid_ids = np.array(icu_data.loc[icu_data.subject_id.isin(valid_sub)].ts_id)
test_ids = np.array(icu_data.loc[icu_data.subject_id.isin(test_sub)].ts_id)

In [17]:
events = events[['ts_id', 'minute', 'variable', 'value', 'table']]
events['value'] = events['value'].astype(float)
events['table'] = events['table'].fillna('N/A')

chunk_size = 10_000_000
agg_results = defaultdict(lambda: {'value_sum': 0, 'value_count': 0, 'table_set': set()})

# First loop: Process the first 6 chunks
for i in tqdm(range(0, chunk_size * 6, chunk_size), desc="Processing First 6 Chunks"):
    chunk = events[i:i + chunk_size]
    
    for row in chunk.itertuples(index=False, name=None):
        key = (row[0], row[1], row[2])  # (ts_id, minute, variable)
        
        agg_results[key]['value_sum'] += row[3]
        agg_results[key]['value_count'] += 1
        agg_results[key]['table_set'].add(row[4])

Processing First 6 Chunks: 100%|██████████| 6/6 [12:00<00:00, 120.09s/it]


In [18]:
# Second loop: Process the remaining chunks
for i in tqdm(range(chunk_size * 6, len(events), chunk_size), desc="Processing Remaining Chunks"):
    chunk = events[i:i + chunk_size]
    
    for row in chunk.itertuples(index=False, name=None):
        key = (row[0], row[1], row[2])  # (ts_id, minute, variable)
        
        agg_results[key]['value_sum'] += row[3]
        agg_results[key]['value_count'] += 1
        agg_results[key]['table_set'].add(row[4])

Processing Remaining Chunks: 100%|██████████| 3/3 [04:44<00:00, 94.84s/it] 


In [19]:
num_parts = 5
keys = list(agg_results.keys())
chunk_size = math.ceil(len(keys) / num_parts)

final_data = []

# Process each part separately
for part in tqdm(range(num_parts), desc="Processing Final Data"):
    start_idx = part * chunk_size 
    end_idx = min((part + 1) * chunk_size, len(keys))
    part_keys = keys[start_idx:end_idx]
    
    for key in part_keys:
        agg = agg_results[key]
        final_data.append([
            key[0], key[1], key[2],                 # ts_id, minute, variable
            agg['value_sum'] / agg['value_count'],  # Mean value
            list(agg['table_set'])                  # Unique tables
        ])
        del key
    del part_keys

Processing Final Data: 100%|██████████| 5/5 [10:02<00:00, 120.41s/it]


In [20]:
final_data

[[37067082.0, 0.0, 'Age', 55.0, ['N/A']],
 [31205490.0, 0.0, 'Age', 46.0, ['N/A']],
 [37510196.0, 0.0, 'Age', 77.0, ['N/A']],
 [39060235.0, 0.0, 'Age', 57.0, ['N/A']],
 [33685454.0, 0.0, 'Age', 81.0, ['N/A']],
 [31090461.0, 0.0, 'Age', 82.0, ['N/A']],
 [32610785.0, 0.0, 'Age', 77.0, ['N/A']],
 [34807493.0, 0.0, 'Age', 81.0, ['N/A']],
 [35479615.0, 0.0, 'Age', 81.0, ['N/A']],
 [33987268.0, 0.0, 'Age', 81.0, ['N/A']],
 [38875437.0, 0.0, 'Age', 81.0, ['N/A']],
 [38392119.0, 0.0, 'Age', 90.0, ['N/A']],
 [35044219.0, 0.0, 'Age', 53.0, ['N/A']],
 [36753294.0, 0.0, 'Age', 81.0, ['N/A']],
 [31831386.0, 0.0, 'Age', 56.0, ['N/A']],
 [37049133.0, 0.0, 'Age', 51.0, ['N/A']],
 [35514836.0, 0.0, 'Age', 64.0, ['N/A']],
 [32128372.0, 0.0, 'Age', 75.0, ['N/A']],
 [34577403.0, 0.0, 'Age', 75.0, ['N/A']],
 [38383343.0, 0.0, 'Age', 75.0, ['N/A']],
 [35796366.0, 0.0, 'Age', 94.0, ['N/A']],
 [35200789.0, 0.0, 'Age', 33.0, ['N/A']],
 [34100191.0, 0.0, 'Age', 47.0, ['N/A']],
 [38292466.0, 0.0, 'Age', 85.0, ['

In [ ]:
import os
import pickle

os.makedirs('./data/final_data', exist_ok=True)

chunk_size = 10000  # Adjust based on memory capacity
for i in range(0, len(final_data), chunk_size):
    chunk = final_data[i:i + chunk_size]
    with open(f'./data/final_data/final_data_part_{i // chunk_size}.pkl', 'wb') as f:
        pickle.dump(chunk, f)

print("Data saved in chunks successfully.")

In [21]:
events = pd.DataFrame(final_data, columns=['ts_id', 'minute', 'variable', 'value', 'table'])
events

,ts_id,minute,variable,value,table
0,37067082.0,0.0,Age,55.0,[N/A]
1,31205490.0,0.0,Age,46.0,[N/A]
2,37510196.0,0.0,Age,77.0,[N/A]
3,39060235.0,0.0,Age,57.0,[N/A]
4,33685454.0,0.0,Age,81.0,[N/A]
...,...,...,...,...,...
80594838,36195440.0,26.0,Weight,60.0,[N/A]
80594839,36195440.0,29.0,Weight,60.0,[N/A]
80594840,36195440.0,39.0,Weight,60.0,[N/A]
80594841,36195440.0,2736.0,Weight,94.0,[N/A]


In [22]:
os.makedirs('./data/events/', exist_ok=True)
icu_data.to_csv('./data/events/events.csv.gz', compression='gzip', index=False)

In [23]:
def f(x):
    if len(x)==0:
        return x[0]
    else:
        return ','.join(x)
events['table'] = events['table'].apply(f)
events

,ts_id,minute,variable,value,table
0,37067082.0,0.0,Age,55.0,N/A
1,31205490.0,0.0,Age,46.0,N/A
2,37510196.0,0.0,Age,77.0,N/A
3,39060235.0,0.0,Age,57.0,N/A
4,33685454.0,0.0,Age,81.0,N/A
...,...,...,...,...,...
80594838,36195440.0,26.0,Weight,60.0,N/A
80594839,36195440.0,29.0,Weight,60.0,N/A
80594840,36195440.0,39.0,Weight,60.0,N/A
80594841,36195440.0,2736.0,Weight,94.0,N/A


In [3]:
os.makedirs('./data/processed', exist_ok=True)
pickle.dump([events, oc, train_ids, valid_ids, test_ids], 
            open('./data/processed/mimic_iv.pkl','wb'))

NameError: name 'events' is not defined

In [3]:
filepath = './data/processed/mimic_iv.pkl'
data, oc, train_ids, val_ids, test_ids = pickle.load(open(filepath, 'rb'))

In [4]:
data

,ts_id,minute,variable,value,table
0,37067082.0,0.0,Age,55.0,N/A
1,31205490.0,0.0,Age,46.0,N/A
2,37510196.0,0.0,Age,77.0,N/A
3,39060235.0,0.0,Age,57.0,N/A
4,33685454.0,0.0,Age,81.0,N/A
...,...,...,...,...,...
80594838,36195440.0,26.0,Weight,60.0,N/A
80594839,36195440.0,29.0,Weight,60.0,N/A
80594840,36195440.0,39.0,Weight,60.0,N/A
80594841,36195440.0,2736.0,Weight,94.0,N/A


In [5]:
oc

,ts_id,hadm_id,subject_id,in_hospital_mortality
0,37067082,24597018,10001217,0
1,31205490,25563031,10001725,0
2,37510196,26184834,10001884,1
3,39060235,23581541,10002013,0
4,33685454,23822395,10002155,0
...,...,...,...,...
57530,37364566,21439025,19999297,1
57531,32336619,26785317,19999442,0
57532,36075953,25744818,19999828,0
57533,38978960,21033226,19999840,1


In [6]:
adm = pd.read_csv(os.path.join(RAW_DATA_PATH,'hosp/admissions.csv.gz'),
                  usecols=['hadm_id', 'subject_id','admittime'])
adm['admittime'] = pd.to_datetime(adm['admittime'])

In [7]:
adm = adm.drop(columns=['subject_id'])
oc_disch = oc[['ts_id', 'hadm_id', 'subject_id']].merge(adm, on='hadm_id', how='left')
oc_disch

,ts_id,hadm_id,subject_id,admittime
0,37067082,24597018,10001217,2157-11-18 22:56:00
1,31205490,25563031,10001725,2110-04-11 15:08:00
2,37510196,26184834,10001884,2131-01-07 20:39:00
3,39060235,23581541,10002013,2160-05-18 07:45:00
4,33685454,23822395,10002155,2129-08-04 12:44:00
...,...,...,...,...
57530,37364566,21439025,19999297,2162-08-14 23:55:00
57531,32336619,26785317,19999442,2148-11-19 10:00:00
57532,36075953,25744818,19999828,2149-01-08 16:44:00
57533,38978960,21033226,19999840,2164-09-10 13:47:00


In [8]:
oc_disch = oc_disch.sort_values(by=['subject_id', 'admittime']).reset_index(drop=True)
oc_disch['group_start_time'] = oc_disch.groupby('subject_id')['admittime'].transform('first')
oc_disch['minutes'] = (oc_disch['admittime'] - oc_disch['group_start_time']).dt.total_seconds() / 60

oc_disch.drop(columns=['group_start_time'], inplace=True)
oc_disch

,ts_id,hadm_id,subject_id,admittime,minutes
0,37067082,24597018,10001217,2157-11-18 22:56:00,0.0
1,31205490,25563031,10001725,2110-04-11 15:08:00,0.0
2,37510196,26184834,10001884,2131-01-07 20:39:00,0.0
3,39060235,23581541,10002013,2160-05-18 07:45:00,0.0
4,33685454,23822395,10002155,2129-08-04 12:44:00,0.0
...,...,...,...,...,...
57530,37364566,21439025,19999297,2162-08-14 23:55:00,0.0
57531,32336619,26785317,19999442,2148-11-19 10:00:00,0.0
57532,36075953,25744818,19999828,2149-01-08 16:44:00,0.0
57533,38978960,21033226,19999840,2164-09-10 13:47:00,0.0


In [13]:
repeated_value = oc_disch['subject_id'][oc_disch['subject_id'].duplicated()]
repeated_value

5        10002155
8        10002428
9        10002428
10       10002428
18       10003400
           ...   
57521    19998330
57522    19998330
57523    19998330
57525    19998591
57529    19999287
Name: subject_id, Length: 15410, dtype: int64

In [15]:
oc_disch[oc_disch.subject_id == 19998330]

,ts_id,hadm_id,subject_id,admittime,minutes
57520,32641669,24492004,19998330,2178-10-01 07:28:00,0.0
57521,38814014,23137777,19998330,2178-10-10 15:23:00,13435.0
57522,31417783,21135114,19998330,2178-10-21 15:14:00,29266.0
57523,33428243,24096838,19998330,2178-11-27 21:51:00,82943.0


In [49]:
oc_disch[oc_disch['subject_id'] == 10000032]

,ts_id,hadm_id,subject_id,admittime,minutes


In [16]:
os.makedirs('./data/processed', exist_ok=True)
pickle.dump([data, oc_disch, train_ids, val_ids, test_ids], 
            open('./data/processed/mimic_iv_disch.pkl','wb'))

In [2]:
_, oc, _, _, _ = pickle.load(open('./data/processed/mimic_iv_disch.pkl', 'rb'))
oc

,ts_id,hadm_id,subject_id,admittime,minutes
0,37067082,24597018,10001217,2157-11-18 22:56:00,0.0
1,31205490,25563031,10001725,2110-04-11 15:08:00,0.0
2,37510196,26184834,10001884,2131-01-07 20:39:00,0.0
3,39060235,23581541,10002013,2160-05-18 07:45:00,0.0
4,33685454,23822395,10002155,2129-08-04 12:44:00,0.0
...,...,...,...,...,...
57530,37364566,21439025,19999297,2162-08-14 23:55:00,0.0
57531,32336619,26785317,19999442,2148-11-19 10:00:00,0.0
57532,36075953,25744818,19999828,2149-01-08 16:44:00,0.0
57533,38978960,21033226,19999840,2164-09-10 13:47:00,0.0


In [3]:
oc[oc.subject_id == 19998330]

,ts_id,hadm_id,subject_id,admittime,minutes
57520,32641669,24492004,19998330,2178-10-01 07:28:00,0.0
57521,38814014,23137777,19998330,2178-10-10 15:23:00,13435.0
57522,31417783,21135114,19998330,2178-10-21 15:14:00,29266.0
57523,33428243,24096838,19998330,2178-11-27 21:51:00,82943.0
